In [1]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *
from dotenv import load_dotenv
import datetime

In [2]:
load_dotenv()
jdbc_driver_path = "postgresql-42.7.4.jar"

In [3]:
spark = SparkSession.builder \
    .appName('Solution') \
    .config("spark.driver.extraClassPath", jdbc_driver_path) \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .getOrCreate()

In [4]:
jdbcHostname = os.getenv("HOST")
jdbcDatabase = os.getenv("DB_NAME")
jdbcUsername = os.getenv("USER")
jdbcPassword = os.getenv("PASSWORD")
jdbcPort = 5432 
jdbcDriver = "org.postgresql.Driver"

connProperties = {
  "user": jdbcUsername,
  "password": jdbcPassword,
  "driver": jdbcDriver
}

jdbcUrl = f"jdbc:postgresql://{jdbcHostname}:{jdbcPort}/{jdbcDatabase}"


In [5]:
work_df = spark.read.jdbc(url=jdbcUrl, table="(SELECT * FROM work)", properties=connProperties)
product_size_df = spark.read.jdbc(url=jdbcUrl, table="(SELECT * FROM product_size)", properties=connProperties)
subject_df = spark.read.jdbc(url=jdbcUrl, table="(SELECT * FROM subject)", properties=connProperties)
artist_df = spark.read.jdbc(url=jdbcUrl, table="(SELECT * FROM artist)", properties=connProperties)
canvas_size_df = spark.read.jdbc(url=jdbcUrl, table="(SELECT * FROM canvas_size)", properties=connProperties)
image_link_df = spark.read.jdbc(url=jdbcUrl, table="(SELECT * FROM image_link)", properties=connProperties)
museum_hours_df = spark.read.jdbc(url=jdbcUrl, table="(SELECT * FROM museum_hours)", properties=connProperties)
museum_df = spark.read.jdbc(url=jdbcUrl, table="(SELECT * FROM museum)", properties=connProperties)

### 1.  Fetch all the paintings which are not displayed on any museums?

In [6]:
not_displayed_paintings = work_df.filter(work_df.museum_id.isNull())
print(not_displayed_paintings.count())
not_displayed_paintings.show()

10223
+-------+--------------------+---------+-------+---------+
|work_id|                name|artist_id|  style|museum_id|
+-------+--------------------+---------+-------+---------+
| 125752|Arabian Horses at...|      757|Baroque|     NULL|
| 125818|Count Halm on His...|      757|Baroque|     NULL|
| 125763|Napoleon Before t...|      757|Baroque|     NULL|
| 125774|Peasants Resting ...|      757|Baroque|     NULL|
| 125785|Portrait Oberleut...|      757|Baroque|     NULL|
| 125796|The Rescue of Cou...|      757|Baroque|     NULL|
| 125807|     The Stable Yard|      757|Baroque|     NULL|
|  24532|Jacob A. Stamler ...|      563|   NULL|     NULL|
| 124470| Kaleda off Le Havre|      563|   NULL|     NULL|
| 124479|R. Bell &amp; Co....|      563|   NULL|     NULL|
| 124488|Steam Sailing Shi...|      563|   NULL|     NULL|
| 124497|The American Ship...|      563|   NULL|     NULL|
| 124506|The Atalanta Runn...|      563|   NULL|     NULL|
| 124515|The Auxiliary Ste...|      563|   NULL|  

### 2.  Are there museums without any paintings?

In [7]:
museum_without_paintings = museum_df.join(work_df, museum_df["museum_id"] == work_df["museum_id"], "left_anti")
print(museum_without_paintings.count())
museum_without_paintings.show()

0
+---------+----+-------+----+-----+------+-------+-----+---+
|museum_id|name|address|city|state|postal|country|phone|url|
+---------+----+-------+----+-----+------+-------+-----+---+
+---------+----+-------+----+-----+------+-------+-----+---+



#### 3. How many paintings have an asking price of more than their regular price?

In [8]:
product_size_df.filter(col("sale_price") \
                       > col("regular_price")).show()

+-------+-------+----------+-------------+
|work_id|size_id|sale_price|regular_price|
+-------+-------+----------+-------------+
+-------+-------+----------+-------------+



#### 4. Identify the paintings whose asking price is less than 50% of its regular price.

In [22]:
filtered_product_size_df = product_size_df.filter(col('sale_price') \
                       < col('regular_price') * 0.5) 

print(filtered_product_size_df.count())
filtered_product_size_df.show()

58
+-------+-------+----------+-------------+
|work_id|size_id|sale_price|regular_price|
+-------+-------+----------+-------------+
|  31780|     36|        10|          125|
|  31780|     30|        10|           95|
|  31780|     36|        10|          125|
|  31780|     30|        10|           95|
| 198417|     36|        30|          125|
| 198417|     30|        30|           95|
|  31974|     24|        30|           85|
|  17351|     24|        10|           85|
|  17351|     30|        10|           95|
|  17351|     36|        10|          125|
|  30947|   3024|       285|          575|
|  30947|   3226|       305|          645|
|  23710|     30|        20|           95|
|  23710|     24|        20|           85|
|  20084|   6040|       585|         1245|
| 133971|#VALUE!|      1025|         2235|
|  28259|     30|        40|           95|
|  28259|     24|        40|           85|
|  28261|     24|        40|           85|
|  28261|     30|        40|           95|
+-------

#### 5. Which canvas size costs the most?

In [16]:
window_spec = Window.orderBy(desc("sale_price"))

ranked_df = product_size_df.withColumn(
    "rnk", 
    rank().over(window_spec)
)

top_ranked_df = ranked_df.filter(
    col("rnk") == 1
)

result_df = top_ranked_df.join(
    canvas_size_df,
    ranked_df["size_id"].cast("string") == canvas_size_df["size_id"],
    "inner"
).select(
    canvas_size_df["label"].alias("canva"), 
    ranked_df["sale_price"]
).show()


+--------------------+----------+
|               canva|sale_price|
+--------------------+----------+
|48" x 96"(122 cm ...|      1115|
+--------------------+----------+



#### 6. Delete duplicate records from the work, product_size, subject, and image_link tables.

#### 7. Identify the museums with invalid city information in the given dataset.

#### 8. Museum_Hours table has 1 invalid entry. Identify it and remove it.

#### 9. Fetch the top 10 most famous painting subjects.

#### 10. Identify the museums which are open on both Sunday and Monday. Display museum name and city.

#### 11. How many museums are open every single day?

#### 12. Which are the top 5 most popular museums? (Popularity is defined based on the most number of paintings in a museum.)

#### 13. Who are the top 5 most popular artists? (Popularity is defined based on the most number of paintings done by an artist.)

#### 14. Display the 3 least popular canvas sizes.

#### 15. Which museum is open for the longest during a day? Display museum name, state, hours open, and which day.

#### 16. Which museum has the most number of paintings in the most popular painting style?

#### 17. Identify the artists whose paintings are displayed in multiple countries.

#### 18. Display the country and the city with the most number of museums. Output two separate columns to mention the city and country. If there are multiple values, separate them with commas.

#### 19. Identify the artist and the museum where the most expensive and least expensive painting is placed. Display the artist's name, sale price, painting name, museum name, museum city, and canvas label.

#### 20. Which country has the 5th highest number of paintings?

#### 21. Which are the 3 most popular and 3 least popular painting styles?

#### 22. Which artist has the most number of portrait paintings outside the USA? Display the artist name, number of paintings, and the artist's nationality.